In [1]:
from matplotlib import pyplot as plt
from numpy import linspace
import numpy as np
import scipy
import scipy.optimize
from cmath import *

# Parameters calculation

In [2]:
#Defenition of all matrices/params
pi = 3.14159265359
hbar = 1.054e-34
fi_0 = 2.07e-15/2/pi

C_J = 0.39e-12
I_J =  2.6e-6
L_J = fi_0 / I_J

print L_J*1e9, 'nH'
print (C_J*L_J)**(-0.5) /6.28e9, 'GHz' 

C_out = 1e-17
C_0 = .41e-15

N = 80
################################

C = []# ^-1
L = []# ^-1

for i in range(N+1):#!!!!!care - goes from 0 to N???
    C.append([])
    L.append([])
    for j in range(N+1):
        if ((i==0)&(j==0)):
            #L[i].append(1./L_J)
            #C[i].append(C_J)
            L[i].append(2./L_J)
            C[i].append(2*C_J + C_0)
            continue
        
        if ((i==N)&(j==N)):
            L[i].append(1./L_J)
            C[i].append(C_J + C_out)
            continue
            
        if (i==j):
            L[i].append(2./L_J)
            C[i].append(2*C_J + C_0)
            continue
            
        if ((j==i+1)or(j==i-1)):
            L[i].append(-1./L_J)
            C[i].append(-C_J)
            continue
        L[i].append(0.)
        C[i].append(0.)

C_test = np.array(C)
C = np.linalg.inv(C)

c_buf = np.diagonal(C)
l_buf = np.diagonal(L)

w = []
for k in range(N+1):
    w.append( (c_buf[k] * l_buf[k])**0.5 )

0.126711820077 nH
22.6516315177 GHz


In [89]:
#Fast check
n = linspace(1, N, N)
fr = []
def fun(k):
    return( (2*L_J / pi**2 * (0.5*C_0*N**2 /k**2 + 0.5* pi**2*C_J ) )**-0.5)

              
plt.plot(n, fun(n)/6.28e9, 'ro')
fr = fun(n)
plt.show()

In [90]:
print fr[0]/6.28e9, fr[1]/6.28e9, fr[2]/6.28e9
print (fr[2] - fr[1])/(fr[1] - fr[0])

17.4672273367 20.9376166084 21.8396017432
0.25990892208


# Luca's calculations

In [88]:
bla  = scipy.linalg.fractional_matrix_power(C_test,-0.5)

(freqs,vs) = np.linalg.eigh(np.dot(bla,np.dot(L,bla)))
                            
nums = linspace(0,N, N + 1)
plt.plot(nums,np.sqrt(freqs)/6.28e9,'ro')
plt.xlim(0,10)
plt.show()

In [87]:
#plt.plot(nums,(vs[79]),'ro')
#plt.show()
print freqs[]**0.5/6.28e9

22.6486543136


## Bogolubov diagonalization

In [96]:
# C_dia = []
# L_dia = []
# C_offDia = []
# L_offDia = []
# for i in range(N+1):
#     C_dia.append([])
#     L_dia.append([])
#     for j in range(N+1):
#         if (i==j):
#             L_dia[i].append(l_buf[i])
#             C_dia[i].append(c_buf[i])
#             continue
        
#         L_dia[i].append(0.)
#         C_dia[i].append(0.)
        
# C_offDia = np.subtract(C,C_dia)
# L_offDia = np.subtract(L,L_dia)

In [4]:
#Now - getting alpha, gamma, M

alpha = []
gamma = []
    
for i in range(N+1):
    alpha.append([])
    gamma.append([])
    for j in range(N+1):#no 2*!!!!!
        alpha[i].append( 0.5 * (w[i]*w[j])**0.5 * ( (l_buf[i]*l_buf[j])**(-0.5) * L[i][j] - (c_buf[i]*c_buf[j])**(-0.5) * C[i][j] ) )
        gamma[i].append( 0.5 * (w[i]*w[j])**0.5 * ( (l_buf[i]*l_buf[j])**(-0.5) * L[i][j] + (c_buf[i]*c_buf[j])**(-0.5) * C[i][j] ) )
        
        
D = np.empty([2*N+2, 2*N+2])

for i in range(2*N+2):
    for j in range(2*N+2):
        if ((i< (N + 1))&(j<(N +1))):
            D[i][j] = (alpha[i][j])
            continue
        
        if ( (i> N )&(j> N) ):
            D[i][j] = (  - alpha[i - N - 1][j - N - 1 ])#-
            continue
            
        if ( (i< (N + 1) )&(j> N ) ):
            D[i][j] = (  gamma[i][j - N - 1] )
            continue    
            
        if ( (i> N  )&(j< (N + 1)) ):
            D[i][j] = (  - gamma[i - N - 1][j] )#-
            continue 

In [5]:
(freq,v) = np.linalg.eigh(D)

In [6]:
nums = linspace(0,2*N + 1, 2*N + 2)
plt.plot(nums - N - 1 ,freq/6.28e9,'ro')
#plt.ylim(-30,30)
plt.show()

In [100]:
nums = linspace(0,2*N + 1, 2*N + 2)
plt.plot(nums - N - 1 ,v[82])
#plt.ylim(-30,30)
plt.show()

# Amplification calc

In [48]:
#Constants


kappa_p = 400e6 * 6.28
kappa_s = 0.66*400e6 * 6.28
kappa_i = 400e6 * 6.28

#w_p0 = 7.5 * 6.28e9
w_s0 = 10.5 * 6.28e9
w_i0 = 3.9 * 6.28e9
w_p0 = (w_s0 + w_i0)/2


K_base = -1e4
K_p =  K_base * 6 *  (w_p0/6.28e9) * 0.25 * 9 * 0.375
K_ps = K_base * 24 * (w_s0/6.28e9)/9 * 25 * 0.25 #No /4!!!!!
K_pi = K_base * 24 * (w_i0/6.28e9) * 0.25
K_psi = K_base * 24* ((w_i0*w_s0)**0.5/6.28e9) / 3 * 5 * 0.125

print K_p/K_base, K_ps/K_base, K_pi/K_base, K_psi/K_base

36.45 175.0 23.4 31.9960935116


Pump duffing

In [47]:
##################################################
#Created to check if power is ok                ##
##################################################
w_p_real = (w_s0 + w_i0)/2


freq_min = w_p_real - 3*kappa_p
freq_max = w_p_real + 3 * kappa_p
freq_num = 5e2


def getPumpAlpha(alpha_in, w_p):
    def f(x,w_p):
        return( (  (w_p - w_p0)**2 + 0.25*kappa_p**2 )*x - 2*(w_p - w_p0)*K_p*x**2 + K_p**2 * x**3 - kappa_p * alpha_in**2 )
    
    a = (scipy.optimize.bisect(lambda x: f(x, w_p), 0,1e9, xtol=1e-17))**0.5
    
    return(a)

###TODO
#check all signs where kappa involved


#Rough search
#for alpha_log in linspace(-30, 30, 10):
 #   alpha_in = 10**(0.1*alpha_log)
    
#Smooth tuning
for alpha_in in linspace(1e6, 2e6, 10):

    def f(x,w_p):
        return( ( (w_p - w_p0)**2 + 0.25*kappa_p**2 )*x - 2*(w_p - w_p0)*K_p*x**2 + K_p**2 * x**3 - kappa_p * alpha_in**2 )

    n_array = []
    for f_p in linspace(freq_min, freq_max, freq_num):
        n_array.append((scipy.optimize.bisect(lambda x: f(x, f_p), 0,1e9, xtol=1e-17))**0.5)

    bufer = np.array(n_array)
    f_p = linspace(freq_min - w_p0, freq_max - w_p0, freq_num)

    plt.plot( (f_p)/kappa_p, bufer, label = str(alpha_in) )
    

#plt.xlim(-2,0)
plt.axvline((w_p_real - w_p0)/6.28e9)
plt.xlabel('(f_p - f_p0)/kappa_p')
plt.ylabel('|alpha|')
plt.title('')
#plt.legend()
plt.show()

In [94]:
getPumpAlpha(5e5, w_p0 - 0.7*kappa_p)

19.163444039389958

In [39]:
def W(w, w_0, kappa ,K, alpha):
    return( 1j * (w - w_0) + 0*1j - 0.5*kappa - 0.5j * K * alpha**2 )

def Gs(ws, w_p,alpha):
    w_i = 2*w_p - w_s
    W_i = W(w_i, w_i0, kappa_i, K_pi, alpha).conjugate()
    W_s = W(w_s, w_s0, kappa_s, K_ps, alpha)
    
    g = -1. - kappa_s * W_i / ( 0*1j + W_s * W_i - 0.25 * K_psi**2 * alpha**4 )
    return(20*np.log10(abs(g)))
    #return g
    
def Gi(ws, w_p,alpha):
    w_i = 2*w_p - w_s
    W_i = W(w_i, w_i0, kappa_i, K_pi, alpha).conjugate()
    W_s = W(w_s, w_s0, kappa_s, K_ps, alpha)

    i = 0.5j * (kappa_s*kappa_i)**0.5 * K_psi*alpha**2/ (0*1j + W_s * W_i - 0.25 * K_psi**2 * alpha**4 )
    return(20*np.log10(abs( i )))
    #return i

In [40]:
#TODO
#Write down code which will plot both Gain(pump parameters) and duffung curves of pump simultaneously!!!!! 

In [45]:
w_p = w_p0 - 0.7*kappa_p
w_s = linspace(w_s0 - 10*kappa_s, w_s0 + 3*kappa_s, 1e4)

#getPumpAlpha(5e5, w_p0 - 0.7*kappa_p)

#for w_p in linspace(w_p0 - 3*kappa_p, w_p0 - 0.5* kappa_p, 10):
w_p = w_p0 - .8*kappa_p
    #alpha = 25.
for w_p in linspace(w_p_real - 1*kappa_p, w_p_real + .5* kappa_p, 10):
    for alpha_in in linspace(1e6,2e6,20):
        #alpha = 10**alpha_pow
        alpha = getPumpAlpha(alpha_in, w_p)
        print alpha
        #print alpha
        plt.plot((w_s - w_s0)/kappa_s,Gs(w_s, w_p, alpha))
        #plt.plot((w_s - w_s0)/kappa_s, abs(Gs(w_s, w_p, alpha))**2 - abs(Gi(w_s, w_p, alpha))**2)
    #plt.ylim(0.,2.)
plt.show()

18.5873222264
19.6622364124
20.75534396
21.8686403452
23.0043996688
24.1652369249
25.3541889763
26.5748212207
27.8313711079
29.1289449426
30.4737948052
31.8737187816
33.3386592207
34.8816331339
36.5202514882
38.2793564649
40.1959785826
42.3297181332
44.7881908199
47.8076894398
21.8506878747
23.1857777732
24.5598799085
25.9789400276
27.4502574402
28.9829689839
30.5887762968
32.2830833563
34.08686605
36.0299395077
38.1571428623
40.5413797878
43.3157438044
46.7751237622
51.909733612
67.5833271582
72.9732132267
75.6059500615
77.4863258658
78.9921501527
26.4390998875
28.2253370027
30.1120836453
32.1234457601
34.2931057103
36.6702314084
39.3305025763
42.3977194984
46.0845871954
50.719105182
56.2578311398
61.1542137646
64.6365490524
67.1891960745
69.1980251841
70.8647613752
72.2991708841
73.5660427781
74.7064398264
75.7479177135
32.9109867323
35.4571264936
38.2152132508
41.2074698802
44.417688902
47.7451209649
50.9929861196
53.9620156822
56.5615020496
58.8060253896
60.7531065023
62.4628055804

In [12]:
#Check
def PumpShifter(alpha):
    return((w_s0 + w_i0 + 0.5 * alpha**2 * (K_ps + K_pi))/2. )

def PumpFreqShift(alpha):
    return(w_p0 + K_p*alpha**2)

alpha = linspace(1e0, 1e2, 5e1)
plt.plot(alpha,(PumpShifter(alpha) - PumpFreqShift(0.)) /kappa_p, 'g', label = 'RealPump')
plt.plot(alpha,(PumpFreqShift(alpha) - PumpFreqShift(0.)) /kappa_p, 'r', label = 'Pump Resonant Freq')
plt.legend(loc = 3)
plt.xlabel('alpha')
plt.ylabel( '(Freq - Freeq_real0) / kappa_p')
plt.axhline(-1.)
plt.show()